In [2]:
from google.colab import files
uploaded = files.upload()

Saving countries.csv to countries.csv
Saving Graph_BACI_HS92_Y2008_V202301.graphml to Graph_BACI_HS92_Y2008_V202301.graphml


In [3]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import numpy as np

class WorldMap:
    def __init__(self):
        self.fig = go.Figure()

    def add_connections(self, connections):
        for connection in connections:
            self.fig.add_trace(
                go.Scattergeo(
                    lon=[connection["lon1"], connection["lon2"]],
                    lat=[connection["lat1"], connection["lat2"]],
                    mode="lines",
                    line=dict(width=connection["weight"], color="red"),
                    opacity=0.6,
                )
            )

    def show(self):
        self.fig.update_layout(
            title="Vínculos entre países",
            showlegend=False,
            geo=dict(
                projection_type="natural earth",
                showland=True,
                landcolor="rgb(243, 243, 243)",
                countrycolor="rgb(204, 204, 204)",
            ),
        )
        self.fig.show()


# Ejemplo de uso
if __name__ == "__main__":
    # Carga la red

    G = nx.read_graphml('Graph_BACI_HS92_Y2008_V202301.graphml')
    filtered_edges = [(u, v, d) for u, v, d in G.edges(data=True) if d["weight"] > 10000000]
    
    # Carga el conjunto de datos de las capitales del mundo
    capitals = pd.read_csv("countries.csv")

    
    # Obtén las coordenadas de cada par de países
    connections = []
    for edge in filtered_edges:
        try:
            country1 = capitals[capitals["id"] == edge[0]].iloc[0]
            country2 = capitals[capitals["id"] == edge[1]].iloc[0]

            connections.append({
                "lat1": country1["latitude"],
                "lon1": country1["longitude"],
                "lat2": country2["latitude"],
                "lon2": country2["longitude"],
                "weight": np.log(edge[2]['weight'])/10,
            })
        except Exception as e:
            continue

    # Crea un objeto WorldMap y añade las conexiones
    world_map = WorldMap()
    world_map.add_connections(connections)

    # Muestra el mapa
    world_map.show()
